In [1]:
import os
import pandas as pd
from tqdm import tqdm

input_path = '/data/wsi/TCTGC10k-features/gigapath-coarse/pt'
output_path = '/data/wsi/TCTGC10k-features/gigapath-700/pt-tmp'
label_path = '/home/huangjialong/projects/BiomedCLIP-PUNCE/datatools/TCTGC10k/9_labels/all_label.csv'

df = pd.read_csv(label_path, header=None)
wsi_names = df[0].tolist()
wsi_names = [wsi_name+'.pt' for wsi_name in wsi_names]

for wsi_name in tqdm(wsi_names):
    if os.path.exists(os.path.join(input_path, wsi_name)):
        os.system(f'cp "{os.path.join(input_path, wsi_name)}" {output_path}')
        # print(os.path.join(input_path, wsi_name))

100%|██████████| 9854/9854 [05:20<00:00, 30.79it/s]


In [5]:
import os
import pandas as pd
from tqdm import tqdm
import torch

input_path = '/data/wsi/TCTGC10k-features/gigapath-1000/pt-tmp'
output_path = '/data/wsi/TCTGC10k-features/gigapath-1000/pt'
keep_same_psize = 1000
wsi_names = os.listdir(input_path)

print(wsi_names)
for wsi_name in tqdm(wsi_names):
    wsi_path = os.path.join(input_path, wsi_name)
    feat = torch.load(wsi_path)

    # 获取当前tensor的长度
    N = feat.shape[0]
    padded_tensor = torch.zeros((keep_same_psize, *(feat.shape[1:]))) 
    # 如果N小于1000，填充数据并设置mask
    if N < keep_same_psize:
        padded_tensor[:N] = feat
    else:  # 如果N大于1000，截断数据并设置mask
        padded_tensor = feat[:keep_same_psize]
    # 保存padded_tensor
    torch.save(padded_tensor, os.path.join(output_path, wsi_name))

['xCR20018105-LSIL.pt', 'C02S2024.pt', 'L2009607.pt', 'L2012883.pt', 'CX20093448.pt', '006234039.pt', 'l2003799.pt', '006233147.pt', '115120744-3-1.pt', 'T230129272.pt', 'T-SZ230312024.pt', 'T220126107.pt', 'T230204164.pt', 'T-SZ2308200011.pt', 'T211231342.pt', '006238323.pt', 'T220527003-NILM.pt', 'T220715055.pt', 'T2310220112.pt', 'T2211240487.pt', 'T221215048.pt', 'CX20176919-ASC-US.pt', 'L2003341.pt', '006238803.pt', 'T221214347.pt', 'T230129230.pt', 'chongqingfy_20231213_22007475.pt', 'L2013345.pt', 'CX20119406.pt', 'CX20164829-ASC-US.pt', 'L2003025.pt', 'L2012582.pt', 'chuxiong20240116_231-2278-US.pt', 'S0292871--.pt', 'chuxiong20231215_US-21.pt', 'L2010070.pt', '002909186.pt', '210201124939-21-1-ASC-US.pt', 'chuxiong20231215_US-9.pt', 'C02S2897.pt', 'T-SZ2308060084.pt', 'T2306220001.pt', 'CX21195593-NILM.pt', 'SXL2017064977 HSIL.pt', 'xCY20006639-NILM.pt', 'A0327438.pt', 'T2306181080.pt', 'CX20110415.pt', 'HBSY-D01-T17-00210.pt', 'l2003806.pt', 'T230131347.pt', 'CX20165663-ASC-U

  0%|          | 0/9854 [00:00<?, ?it/s]

100%|██████████| 9854/9854 [39:07<00:00,  4.20it/s]  


# 预先进行聚类

In [16]:
import os
import pandas as pd
import torch
from tqdm import tqdm
from cuml.cluster import KMeans
import cupy as cp

# from sklearn.cluster import KMeans

# Your existing setup
input_path = '/data/wsi/TCTGC10k-features/gigapath-1000/pt-tmp'
input_label = '/data/wsi/TCTGC10k-labels/6_labels/TCTGC10k-v15-train.csv'
labels = pd.read_csv(input_label)
n_clusters = 5
output_path = './cluster'

# Create DataFrame with WSI names
df = labels[['wsi_name']].copy()
df['cluster_label'] = None  # Add column for storing pooled features

# Process each WSI
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing WSIs"):
    wsi_name = row['wsi_name']
    pt_path = os.path.join(input_path, f"{wsi_name}.pt")
    
    try:
        features = torch.load(pt_path)
        # Load features
        # kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        # cluster_labels = kmeans.fit_predict(features)

        patch_features_cp = cp.asarray(features)
        # patch_features_cp = patch_features
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        cluster_labels = kmeans.fit_predict(patch_features_cp)
        df.at[idx, 'cluster_label'] = " ".join(map(str, cluster_labels))
        
    except Exception as e:
        print(f"Error processing {wsi_name}: {str(e)}")
        df.at[idx, 'cluster_label'] = None  # Mark as failed

# Optional: Save the DataFrame with pooled features
output_df_path = os.path.join(output_path, f"kmeans_{n_clusters}.csv")
df.to_csv(output_df_path, index=False)
print(f"Saved pooled features to {output_df_path}")

Processing WSIs:   0%|          | 0/6780 [00:00<?, ?it/s]

/tmp/ipykernel_25114/25750672.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  features = torch.load(pt_path)
Processing WSIs: 100%|██████████| 6780/6780 [08:34<00:00, 1

Saved pooled features to ./cluster/kmeans_5.csv


In [17]:
import pandas as pd
import torch
import os

input_file = '/home/huangjialong/projects/BiomedCLIP-PUNCE/datatools/TCTGC10k/cluster/kmeans_5.csv'
feat_dir = '/data/wsi/TCTGC50k-features/gigapath-coarse/pt'
df = pd.read_csv(input_file)
for i in range(len(df)):
    wsi_name = df.iloc[i, 0]
    cluster_label = [int(x) for x in df.iloc[i, 1].split(' ')]
    feat_path = os.path.join(feat_dir, wsi_name + '.pt')
    feat = torch.load(feat_path)
    print(feat.shape, len(cluster_label))
    

/tmp/ipykernel_25114/2530657942.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  feat = torch.load(feat_path)


torch.Size([442, 1536]) 442
torch.Size([475, 1536]) 475
torch.Size([588, 1536]) 588
torch.Size([520, 1536]) 520
torch.Size([468, 1536]) 468
torch.Size([864, 1536]) 864
torch.Size([768, 1536]) 768
torch.Size([1440, 1536]) 1440
torch.Size([736, 1536]) 736
torch.Size([513, 1536]) 513
torch.Size([1440, 1536]) 1440
torch.Size([580, 1536]) 580
torch.Size([759, 1536]) 759
torch.Size([693, 1536]) 693
torch.Size([580, 1536]) 580
torch.Size([850, 1536]) 850
torch.Size([494, 1536]) 494
torch.Size([580, 1536]) 580
torch.Size([792, 1536]) 792
torch.Size([442, 1536]) 442
torch.Size([513, 1536]) 513
torch.Size([540, 1536]) 540
torch.Size([588, 1536]) 588
torch.Size([588, 1536]) 588
torch.Size([540, 1536]) 540
torch.Size([425, 1536]) 425
torch.Size([759, 1536]) 759
torch.Size([759, 1536]) 759
torch.Size([580, 1536]) 580
torch.Size([520, 1536]) 520
torch.Size([459, 1536]) 459
torch.Size([468, 1536]) 468
torch.Size([459, 1536]) 459
torch.Size([532, 1536]) 532
torch.Size([540, 1536]) 540
torch.Size([450,

KeyboardInterrupt: 